In [1]:
#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
from qiskit.quantum_info import partial_trace, Statevector

from numpy import pi

In [2]:
# a >= b

def greaterThanEqual(qc, a, b, aux, output, n):
        
    # Check most significant bit

    # Check that a[n-1] is 1 and b[n-1] is 0
    qc.x(b[n-1])
    qc.ccx(a[n-1], b[n-1], output[0])
    
    # Check if both a[n-1] and b[n-1] are 0 
    qc.x(a[n-1])
    qc.ccx(a[n-1], b[n-1], aux[n-1])
    
    # Check if both a[n-1] and b[n-1] are 1
    qc.x(a[n-1])
    qc.x(b[n-1])
    qc.ccx(a[n-1], b[n-1], aux[n-1])
    
    # Check the rest of the bits
    for i in range(n-2, -1, -1):
        # Check that a[i] is 1 and b[i] is 0 if the previous bits were equal
        qc.x(b[i])
        qc.mct([a[i], b[i], aux[i+1]], output[0])
    
        # Check if both a[i] and b[i] are 0 
        qc.x(a[i])
        qc.mct([a[i], b[i], aux[i+1]], aux[i])
    
        # Check if both a[i] and b[i] are 1
        qc.x(a[i])
        qc.x(b[i])
        qc.mct([a[i], b[i], aux[i+1]], aux[i])
        
    # Returning 1 if both numbers are equal
    qc.cx(aux[0], output[0])
    
    # Reversing circuit to put aux back to original state
    for i in range(0, n-1):
    
        # Check if both a[i] and b[i] are 0
        qc.x(b[i])
        qc.x(a[i])
        qc.mct([a[i], b[i], aux[i+1]], aux[i])
    
        # Check if both a[i] and b[i] are 1
        qc.x(a[i])
        qc.x(b[i])
        qc.mct([a[i], b[i], aux[i+1]], aux[i])
        
    qc.x(b[n-1])
    qc.x(a[n-1])
    qc.ccx(a[n-1], b[n-1], aux[n-1])
    
    # Check if both a[n-1] and b[n-1] are 1
    qc.x(a[n-1])
    qc.x(b[n-1])
    qc.ccx(a[n-1], b[n-1], aux[n-1])
    
def initilizeQubits(qc, b, regB, n):
    for i in range(n):
        if (b[i] == "1"):
            qc.x(regB[n-(i+1)])
            
def comparisonOracle(qc, a, b, aux0, aux1, output, n):
    # Compute
    greaterThanEqual(qc, a, b, aux0, aux1, n)
    
    qc.cx(aux1, output)
    
    # Uncompute
    greaterThanEqual(qc, a, b, aux0, aux1, n)
    
def diffuser(a, nqubits):
#     qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(a[qubit])
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(a[qubit])
    # Do multi-controlled-Z gate
    qc.h(a[nqubits-1])
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s

In [14]:
a = "01"
b = "1"

n = 1
regA = QuantumRegister(n, name="a")
regB = QuantumRegister(n, name="b")
regAux = QuantumRegister(n, name="aux")
regAux2 = QuantumRegister(1, name="aux2")
regO = QuantumRegister(1, name="o")

output = ClassicalRegister(n)

qc = QuantumCircuit(regA, regB, regAux, regAux2, regO, output)

# Turning output to state |->
qc.x(regO)
qc.h(regO)
initilizeQubits(qc, b, regB, n)
qc.h(regA)
comparisonOracle(qc, regA, regB, regAux, regAux2, regO, n)
qc.barrier()
# qc.append(diffuser(n), regA)
# qc.measure(regA, output)
qc.draw()

┌───┐          ┌───┐     ┌───┐     ┌───┐     ┌───┐               ┌───┐»
   a: ┤ H ├───────■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■─────────■──┤ X ├»
      ├───┤┌───┐  │  └───┘  │  ├───┤  │  ├───┤  │  ├───┤  │  ┌───┐  │  └───┘»
   b: ┤ X ├┤ X ├──■─────────■──┤ X ├──■──┤ X ├──■──┤ X ├──■──┤ X ├──■───────»
      └───┘└───┘  │       ┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐└───┘  │       »
 aux: ────────────┼───────┤ X ├─────┤ X ├──■──┤ X ├─────┤ X ├───────┼───────»
                ┌─┴─┐     └───┘     └───┘┌─┴─┐└───┘     └───┘     ┌─┴─┐     »
aux2: ──────────┤ X ├────────────────────┤ X ├──■─────────────────┤ X ├─────»
      ┌───┐┌───┐└───┘                    └───┘┌─┴─┐               └───┘     »
   o: ┤ X ├┤ H ├──────────────────────────────┤ X ├─────────────────────────»
      └───┘└───┘                              └───┘                         »
c5: 1/══════════════════════════════════════════════════════════════════════»
                                                                            »
«           ┌───┐     ┌───┐     ┌───┐      ░ 
«   a: ──■──┤ X ├──■──┤ X ├──■──┤ X ├──■───░─
«        │  ├───┤  │  ├───┤  │  ├───┤  │   ░ 
«   b: ──■──┤ X ├──■──┤ X ├──■──┤ X ├──■───░─
«      ┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐ ░ 
« aux: ┤ X ├─────┤ X ├──■──┤ X ├─────┤ X ├─░─
«      └───┘     └───┘┌─┴─┐└───┘     └───┘ ░ 
«aux2: ───────────────┤ X ├────────────────░─
«                     └───┘                ░ 
«   o: ────────────────────────────────────░─
«                                          ░ 
«c5: 1/══════════════════════════════════════
«

In [15]:
# Measure
aer_sim = Aer.get_backend('aer_simulator')
# transpiled_grover_circuit = transpile(qc, aer_sim)
# transpiled_grover_circuit.save_statevector()
qc.save_statevector()
qobj = assemble(qc)
result = aer_sim.run(qobj).result()
statevec = result.get_statevector()
# partial_density_matrix = partial_trace(statevec, [1, 2])

# # extract the statevector out of the density matrix
# partial_statevector = np.diagonal(partial_density_matrix)
# print(partial_statevector)
from qiskit_textbook.tools import vector2latex
vector2latex(statevec, pretext="|\\psi\\rangle =")

<IPython.core.display.Math object>

In [113]:
pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src

  Cloning https://github.com/qiskit-community/qiskit-textbook.git to c:\users\john\appdata\local\temp\pip-req-build-gqizw0a4
  Resolved https://github.com/qiskit-community/qiskit-textbook.git to commit 06aa460380f30e7101143d27a545e0829970a75f
  Created wheel for qiskit-textbook: filename=qiskit_textbook-0.1.0-py3-none-any.whl size=17058 sha256=8e1b05d6ee52f60cc2d32cede3caf717bef83378386de12352a176ef69c2ddb6
  Stored in directory: C:\Users\John\AppData\Local\Temp\pip-ephem-wheel-cache-jlvpts_0\wheels\75\76\cd\ff92751b9ab5f2d12c73e551ee92adf4615fb6b7d37b77f0e4
Successfully built qiskit-textbook
Note: you may need to restart the kernel to use updated packages.


  Running command git clone -q https://github.com/qiskit-community/qiskit-textbook.git 'C:\Users\John\AppData\Local\Temp\pip-req-build-gqizw0a4'
